<a href="https://colab.research.google.com/github/anisaAnya/Kaggle_competitions/blob/master/Digit_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
#learn from https://www.kaggle.com/juiyangchang/cnn-with-pytorch-0-995-accuracyport numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [0]:
!pip install torch torchvision

In [0]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [0]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid

import math
import random

from PIL import Image, ImageOps, ImageEnhance
import numbers

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
train_df=pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
class mydata(Dataset):
     def __init__(self, file_path, 
                 transform = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(), 
                     transforms.Normalize(mean=(0.5,), std=(0.5,))])
                ):
        
        df = pd.read_csv(file_path)
        
        if len(df.columns) == n_pixels:
            # test data
            self.X = df.values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = None
        else:
            # training data
            self.X = df.iloc[:,1:].values.reshape((-1,28,28)).astype(np.uint8)[:,:,:,None]
            self.y = torch.from_numpy(df.iloc[:,0].values)
            
        self.transform = transform
    
     def __len__(self):
        return len(self.X)

     def __getitem__(self, idx):
        if self.y is not None:
            return self.transform(self.X[idx]), self.y[idx]
        else:
            return self.transform(self.X[idx])

In [0]:
class RandomShift(object):
    def __init__(self, shift):
        self.shift = shift
        
    @staticmethod
    def get_params(shift):
        """Get parameters for ``rotate`` for a random rotation.
        Returns:
            sequence: params to be passed to ``rotate`` for random rotation.
        """
        hshift, vshift = np.random.uniform(-shift, shift, size=2)

        return hshift, vshift 
    def __call__(self, img):
        hshift, vshift = self.get_params(self.shift)
        
        return img.transform(img.size, Image.AFFINE, (1,0,hshift,0,1,vshift), resample=Image.BICUBIC, fill=1)
    

In [0]:
batch_size = 64
n_test = len(test_df)
n_pixels = len(test_df.columns)
train_dataset = mydata('train.csv', transform= transforms.Compose(
                            [transforms.ToPILImage(),transforms.RandomRotation(degrees=40), RandomShift(4),
                             transforms.ToTensor(), transforms.Normalize(mean=(0.5,), std=(0.5,))]))
test_dataset = mydata('test.csv', )


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size, shuffle=False)

In [0]:
class Flatten(nn.Module):

    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        shape = torch.prod(torch.tensor(x.shape[1:])).item()
        return x.view(-1, shape)

In [0]:
class Dense(nn.Module):

    def __init__(self,input_n,out_n):
        super(Dense, self).__init__()

        # Inputs = 5, Outputs = 3, Hidden = 30
        self.fc1 = nn.Linear(input_n, 128)
        self.fc2 = nn.Linear(128, out_n)

    def forward(self, state):
        x = F.relu(self.fc1(state))
        outputs = self.fc2(x)
        return outputs


In [0]:
class Net(nn.Module):    
    def __init__(self):
        super(Net, self).__init__()
        main = nn.Sequential()
        main.add_module('Flatten', Flatten())
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p = 0.25),
            
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(p = 0.25),
            
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.Dropout(p = 0.25),
            
        )
          
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.25),
            nn.Linear(1600, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.25),
            nn.Linear(512, 10),
            nn.Dropout(p = 0.25),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1) 
        x = self.classifier(x)
        
        return x     

In [0]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [0]:
print(model)

Net(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU(inplace)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.25)
    (8): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (9): ReLU(inplace)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Dropout(p=0.25

In [0]:
def train(epoch):
    model.train()
    exp_lr_scheduler.step()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        #print(loss.data)
        
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader),
                loss.item()))

In [0]:
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target, size_average=False).item() 

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
       loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

In [0]:
n_epochs = 5

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [6400/42000 (15%)]	Loss: 0.787041
Train Epoch: 0 [12800/42000 (30%)]	Loss: 0.689215
Train Epoch: 0 [19200/42000 (46%)]	Loss: 0.757866
Train Epoch: 0 [25600/42000 (61%)]	Loss: 0.434492
Train Epoch: 0 [32000/42000 (76%)]	Loss: 0.535299
Train Epoch: 0 [38400/42000 (91%)]	Loss: 0.605747


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Average loss: 0.1755, Accuracy: 39822/42000 (94.000%)

Train Epoch: 1 [6400/42000 (15%)]	Loss: 0.523207
Train Epoch: 1 [12800/42000 (30%)]	Loss: 0.288132
Train Epoch: 1 [19200/42000 (46%)]	Loss: 0.356521
Train Epoch: 1 [25600/42000 (61%)]	Loss: 0.362271
Train Epoch: 1 [32000/42000 (76%)]	Loss: 0.466572
Train Epoch: 1 [38400/42000 (91%)]	Loss: 0.447510

Average loss: 0.1139, Accuracy: 40649/42000 (96.000%)

Train Epoch: 2 [6400/42000 (15%)]	Loss: 0.203960
Train Epoch: 2 [12800/42000 (30%)]	Loss: 0.399571
Train Epoch: 2 [19200/42000 (46%)]	Loss: 0.324025
Train Epoch: 2 [25600/42000 (61%)]	Loss: 0.508818
Train Epoch: 2 [32000/42000 (76%)]	Loss: 0.307015
Train Epoch: 2 [38400/42000 (91%)]	Loss: 0.325869

Average loss: 0.0821, Accuracy: 41047/42000 (97.000%)

Train Epoch: 3 [6400/42000 (15%)]	Loss: 0.249759
Train Epoch: 3 [12800/42000 (30%)]	Loss: 0.297472
Train Epoch: 3 [19200/42000 (46%)]	Loss: 0.334838
Train Epoch: 3 [25600/42000 (61%)]	Loss: 0.308577
Train Epoch: 3 [32000/42000 (76%)]	

In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.000003)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.005)

n_epochs = 7

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [6400/42000 (15%)]	Loss: 0.351474
Train Epoch: 0 [12800/42000 (30%)]	Loss: 0.314276
Train Epoch: 0 [19200/42000 (46%)]	Loss: 0.293155
Train Epoch: 0 [25600/42000 (61%)]	Loss: 0.408353
Train Epoch: 0 [32000/42000 (76%)]	Loss: 0.481477
Train Epoch: 0 [38400/42000 (91%)]	Loss: 0.227153


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Average loss: 0.0656, Accuracy: 41270/42000 (98.000%)

Train Epoch: 1 [6400/42000 (15%)]	Loss: 0.268242
Train Epoch: 1 [12800/42000 (30%)]	Loss: 0.175407
Train Epoch: 1 [19200/42000 (46%)]	Loss: 0.388101
Train Epoch: 1 [25600/42000 (61%)]	Loss: 0.351373
Train Epoch: 1 [32000/42000 (76%)]	Loss: 0.358885
Train Epoch: 1 [38400/42000 (91%)]	Loss: 0.322900

Average loss: 0.0603, Accuracy: 41290/42000 (98.000%)

Train Epoch: 2 [6400/42000 (15%)]	Loss: 0.270314
Train Epoch: 2 [12800/42000 (30%)]	Loss: 0.383709
Train Epoch: 2 [19200/42000 (46%)]	Loss: 0.563525
Train Epoch: 2 [25600/42000 (61%)]	Loss: 0.394895
Train Epoch: 2 [32000/42000 (76%)]	Loss: 0.165169
Train Epoch: 2 [38400/42000 (91%)]	Loss: 0.384146

Average loss: 0.0607, Accuracy: 41284/42000 (98.000%)

Train Epoch: 3 [6400/42000 (15%)]	Loss: 0.409781
Train Epoch: 3 [12800/42000 (30%)]	Loss: 0.456798
Train Epoch: 3 [19200/42000 (46%)]	Loss: 0.375175
Train Epoch: 3 [25600/42000 (61%)]	Loss: 0.373437
Train Epoch: 3 [32000/42000 (76%)]	

In [0]:
def prediciton(data_loader):
    model.eval()
    test_pred = torch.LongTensor()
    
    for i, data in enumerate(data_loader):
        data = Variable(data, volatile=True)
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred
test_pred = prediciton(test_loader)
out_df = pd.DataFrame(np.c_[np.arange(1, len(test_dataset)+1)[:,None], test_pred.numpy()], 
                      columns=['ImageId', 'Label'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [0]:
out_df.to_csv('submission111.csv', index=False)